In [24]:
import numpy as np
import os
import time
folder = "Problems"

time_limit = 300
k = 2

In [25]:
def getpermutation(data, n):
    cur_ans = []
    if data == []:
        return [[]]
    for i in range(len(data)):
        last_answer = getpermutation(data[:i]+data[i+1:], n)
        
        for j in range(len(last_answer)): 
            last_answer[j].append(data[i])

        cur_ans.extend(last_answer)
    if len(data) == n:
        cur_ans.pop()
    return cur_ans

In [26]:
def getpermutation2(start, end, k):
    if k == 0:
        return [[]]
    ans = []
    for i in range(start,end-(k-1)):
        last_ans = getpermutation2(i+1, end,k-1)
        for j in last_ans:
            ans.append([i] + j)
    return ans

In [27]:
result = [[] for _ in os.listdir(folder)]
for step, dirs in enumerate(sorted(os.listdir(folder), key = lambda x: int(x))):
    print("Test with number of {} begin!".format(dirs))
    cur_dir = os.path.join(folder, dirs)
    for files in os.listdir(cur_dir):
        start_time = time.time()
        path = os.path.join(cur_dir, files)
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read().split()
            data = list(map(float, content))
            number = int(data[0])
            distances = np.zeros((number,number))
        for i in range(number):
            for j in range(number):
                distances[i][j] = data[i*number+j+1]
        
        ans = np.inf
        ans_route = [""]
        choices = getpermutation2(1, number, k)
        for _ in range(10000):
            if time.time() - start_time > time_limit:
                break

            order = list(np.arange(1, number))
            np.random.shuffle(order)
            route = np.array([0] + order + [0])
            
            while True:
                if time.time() - start_time > time_limit:
                    break
                improvement = 0
                for choice in choices:
                    if time.time() - start_time > time_limit:
                        break
                    clusters = [[0]]
                    for i in range(1, len(choice)):
                        if choice[i-1]+1 == choice[i]:
                            clusters[-1].append(i)
                        else:
                            clusters.append([i])
                    current = 0
                    for cluster in clusters:
                        # print(distances[route[choice[cluster[0]]]][route[choice[cluster[0]]-1]] + distances[route[choice[cluster[-1]]]][route[choice[cluster[-1]]+1]])
                        current += distances[route[choice[cluster[0]]]][route[choice[cluster[0]]-1]] + distances[route[choice[cluster[-1]]]][route[choice[cluster[-1]]+1]]
                        for i in range(1, len(cluster)):
                            current += distances[route[choice[cluster[i]]]][route[choice[cluster[i-1]]]]
                    for permutation in getpermutation(choice, len(choice)):
                        next = 0
                        for cluster in clusters:
                            next += distances[route[permutation[cluster[0]]]][route[choice[cluster[0]]-1]] + distances[route[permutation[cluster[-1]]]][route[choice[cluster[-1]]+1]]
                            for i in range(1, len(cluster)):
                                next += distances[route[permutation[cluster[i]]]][route[permutation[cluster[i-1]]]]
                        if current - next > improvement:
                            next_step = (choice, permutation)
                            improvement = current - next
                if improvement < 10**(-6):
                    break
                odd_route = list(route)
                route[next_step[0]] = route[next_step[1]]
                # print(next_step)
                # print(odd_route, route)
                # print(improvement, sum([distances[odd_route[i]][odd_route[i+1]] for i in range(number)])-sum([distances[route[i]][route[i+1]] for i in range(number)]))
            cur = sum([distances[route[i]][route[i+1]] for i in range(number)])
            if cur < ans:
                ans = cur   
                ans_route = route 
               
        print("ans is: {}, route is: {}".format(np.round(ans, 2), '->'.join(map(str, ans_route))))
        result[step].append(ans)
    print()
    print()



Test with number of 10 begin!
ans is: 469.78, route is: 0->6->5->4->2->8->3->9->1->7->0
ans is: 449.22, route is: 0->5->4->2->7->9->3->8->6->1->0
ans is: 416.75, route is: 0->3->9->1->7->5->8->6->4->2->0
ans is: 506.28, route is: 0->2->6->4->5->7->1->3->8->9->0
ans is: 518.05, route is: 0->9->2->8->5->1->3->4->7->6->0
ans is: 420.27, route is: 0->3->7->6->5->9->4->8->2->1->0
ans is: 547.28, route is: 0->7->6->1->4->2->8->3->5->9->0
ans is: 484.33, route is: 0->7->5->3->4->1->8->9->6->2->0
ans is: 735.16, route is: 0->6->9->8->5->4->7->3->2->1->0
ans is: 470.02, route is: 0->8->1->6->4->5->3->9->7->2->0


Test with number of 20 begin!
ans is: 752.44, route is: 0->12->7->11->3->2->19->8->14->5->4->10->17->15->1->13->6->16->9->18->0
ans is: 646.79, route is: 0->15->4->6->10->17->13->3->9->14->18->12->11->16->5->2->7->19->1->8->0
ans is: 742.28, route is: 0->7->12->18->16->10->11->3->13->8->5->2->15->1->17->6->14->19->4->9->0
ans is: 455.43, route is: 0->7->5->17->14->4->19->2->3->15->10->

In [21]:
from openpyxl import load_workbook

# 载入已存在的xlsx文件
wb = load_workbook('result.xlsx')

# 选择一个工作表
ws = wb["300s"]

# 写入数据到第A列（你可以更改列名来选择不同的列）
ws[f'G{1}'] = "2_points"
for i, item in enumerate(result, start=2):  # start=1 表示从第一行开始
    ws[f'G{i}'] = np.mean(item)
# 保存文件
wb.save('result.xlsx')